In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
event_weight = {'dislike':-10, 'skip':-0.1, 'view':0.1, 'like':0.5}

In [7]:
customer_train = pd.read_csv('customer_train.csv')
customer_train.head()

,customer_id,product_0,product_1,product_2,product_3,product_4,product_5,product_6,gender_cd,age,marital_status_cd,children_cnt,first_session_dttm,job_position_cd,job_title
0,894436,NaN,NaN,NaN,NaN,NaN,UTL,NaN,M,30.0,MAR,0.0,2018-03-20 09:10:16,1,Неруководящий сотрудник - обсл. Персонал
1,524526,NaN,UTL,NaN,NaN,NaN,UTL,NaN,F,20.0,UNM,0.0,2017-03-29 20:38:45,16,NaN
2,498134,NaN,UTL,NaN,NaN,NaN,NaN,NaN,F,25.0,UNM,0.0,2018-03-12 11:25:06,22,NaN
3,278941,NaN,NaN,UTL,CLS,NaN,UTL,UTL,M,25.0,NaN,NaN,2016-02-21 18:47:51,16,Неруководящий сотрудник - специалист
4,877312,NaN,UTL,NaN,NaN,NaN,NaN,NaN,F,40.0,MAR,0.0,2018-03-07 11:17:02,22,NaN


In [15]:
customer_train = pd.read_csv('customer_train.csv')

# for products: new value for NaN - NU == not used
customer_train[['product_0', 
                'product_1', 
                'product_2', 
                'product_3', 
                'product_4', 
                'product_5', 
                'product_6']] = customer_train[['product_0', 
                                                'product_1', 
                                                'product_2', 
                                                'product_3', 
                                                'product_4', 
                                                'product_5', 
                                                'product_6']].fillna(value='NU')

# for children: if NaN -> let it be 0.0 !May be wrong direction
#customer_train['children_cnt'].fillna(value=0.0, inplace=True)

# for marital status: if Nan -> let it be UNM !May be wrong direction
customer_train['marital_status_cd'].fillna(value='UNM', inplace=True)

# for gender_cd: one method for filling
#customer_train['gender_cd'].fillna(method='ffill', inplace=True)

# for age: mean(customer_train['age'])
#customer_train['age'].fillna(value=customer_train.age.mean(), inplace=True)

# for job_title: drop column for a while
customer_train.drop(columns='job_title', inplace=True)

customer_train.dropna(inplace=True)
print(len(customer_train))
customer_train.head()

44716


,customer_id,product_0,product_1,product_2,product_3,product_4,product_5,product_6,gender_cd,age,marital_status_cd,children_cnt,first_session_dttm,job_position_cd
0,894436,NU,NU,NU,NU,NU,UTL,NU,M,30.0,MAR,0.0,2018-03-20 09:10:16,1
1,524526,NU,UTL,NU,NU,NU,UTL,NU,F,20.0,UNM,0.0,2017-03-29 20:38:45,16
2,498134,NU,UTL,NU,NU,NU,NU,NU,F,25.0,UNM,0.0,2018-03-12 11:25:06,22
4,877312,NU,UTL,NU,NU,NU,NU,NU,F,40.0,MAR,0.0,2018-03-07 11:17:02,22
5,821806,NU,NU,NU,UTL,NU,UTL,NU,F,25.0,UNM,0.0,2018-01-15 09:23:54,22


In [2]:
transaction = pd.read_csv('transactions.csv')#[['customer_id', 'transaction_amt']]
transaction.head()

,customer_id,transaction_month,transaction_day,transaction_amt,merchant_id,merchant_mcc
0,855115,7,3,1500,4554547,5411
1,997036,6,6,0,1657528,5411
2,398237,5,24,2500,26375569,5813
3,997036,6,2,0,16304402,5411
4,291636,7,25,0,1259505,5411


In [4]:
len(set(transaction['merchant_id']))

664198

In [ ]:
customer_train = customer_train.merge(transaction, how='left', by='customer_id')
customer.dropna()

### Categorical features

In [16]:
categorical = pd.get_dummies(customer_train[['product_0',
                                             'product_1',
                                             'product_2',
                                             'product_3',
                                             'product_4',
                                             'product_5',
                                             'product_6',
                                             'gender_cd',
                                             'marital_status_cd']])
categorical['customer_id'] = customer_train['customer_id']
categorical.head()

,product_0_CLS,product_0_NU,product_0_UTL,product_1_CLS,product_1_NU,product_1_OPN,product_1_UTL,product_2_CLS,product_2_NU,product_2_OPN,...,product_6_UTL,gender_cd_F,gender_cd_M,marital_status_cd_CIV,marital_status_cd_DIV,marital_status_cd_DLW,marital_status_cd_MAR,marital_status_cd_UNM,marital_status_cd_WID,customer_id
0,0,1,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,894436
1,0,1,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,524526
2,0,1,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,498134
4,0,1,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,1,0,0,877312
5,0,1,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,821806


In [17]:
categorical_grouped = categorical.groupby('customer_id').agg('sum')
categorical_grouped.head()

,product_0_CLS,product_0_NU,product_0_UTL,product_1_CLS,product_1_NU,product_1_OPN,product_1_UTL,product_2_CLS,product_2_NU,product_2_OPN,...,product_6_OPN,product_6_UTL,gender_cd_F,gender_cd_M,marital_status_cd_CIV,marital_status_cd_DIV,marital_status_cd_DLW,marital_status_cd_MAR,marital_status_cd_UNM,marital_status_cd_WID
customer_id,,,,,,,,,,,,,,,,,,,,,
15,0,1,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
73,1,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
91,0,1,0,1,0,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,0
129,0,1,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,1,0,0
144,0,1,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,1,0,0


### Data type

In [18]:
datetime = customer_train[['customer_id', 'first_session_dttm']]

datetime['first_session_dttm'] = pd.to_datetime(datetime['first_session_dttm'])
earliest_date = datetime['first_session_dttm'].min()

datetime['seconds'] = (datetime['first_session_dttm'] - earliest_date)/np.timedelta64(1,'s')
date_mean = np.mean(datetime['seconds'])
date_std = np.std(datetime['seconds'])
datetime['seconds'] = (datetime['seconds'] - date_mean)/date_std

datetime.drop(columns='first_session_dttm', inplace=True)

datetime.head()

,customer_id,seconds
0,894436,0.877672
1,524526,0.184516
2,498134,0.862257
4,877312,0.852498
5,821806,0.752910


### Numerical data

In [19]:
numerical = customer_train[['age', 'children_cnt', 'job_position_cd']]
for column in numerical.columns:
    numerical[column] = (numerical[column] - np.mean(numerical[column]))/np.std(numerical[column])
numerical['customer_id'] = customer_train['customer_id']
    
numerical.head()

,age,children_cnt,job_position_cd,customer_id
0,-0.392605,-0.403651,-2.324087,894436
1,-1.432667,-0.403651,0.000125,524526
2,-0.912636,-0.403651,0.929810,498134
4,0.647457,-0.403651,0.929810,877312
5,-0.912636,-0.403651,0.929810,821806


### Merge this parts of data

In [20]:
customer_train_merged = categorical_grouped.merge(datetime, how='left', on='customer_id')
customer_train_merged = customer_train_merged.merge(numerical, how='left', on='customer_id')

customer_train_merged.describe()

,customer_id,product_0_CLS,product_0_NU,product_0_UTL,product_1_CLS,product_1_NU,product_1_OPN,product_1_UTL,product_2_CLS,product_2_NU,...,marital_status_cd_CIV,marital_status_cd_DIV,marital_status_cd_DLW,marital_status_cd_MAR,marital_status_cd_UNM,marital_status_cd_WID,seconds,age,children_cnt,job_position_cd
count,44716.000000,44716.000000,44716.000000,44716.000000,44716.000000,44716.000000,44716.000000,44716.000000,44716.000000,44716.000000,...,44716.000000,44716.000000,44716.000000,44716.000000,44716.000000,44716.000000,4.471600e+04,4.471600e+04,4.471600e+04,4.471600e+04
mean,507336.209925,0.000380,0.964644,0.034976,0.042334,0.483339,0.029385,0.444941,0.015856,0.910144,...,0.049401,0.056199,0.000022,0.309822,0.578294,0.006262,-2.644116e-16,3.368706e-16,-4.067871e-17,-6.991654e-18
std,274948.939081,0.019495,0.184681,0.183722,0.201352,0.499728,0.168886,0.496965,0.124918,0.285979,...,0.216706,0.230308,0.004729,0.462425,0.493838,0.078884,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00
min,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.242666e+00,-1.952698e+00,-4.036508e-01,-2.324087e+00
25%,284950.750000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.776107e-01,-9.126359e-01,-4.036508e-01,1.247453e-04
50%,513738.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.030426e-01,-3.926050e-01,-4.036508e-01,1.247453e-04
75%,742962.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,7.183498e-01,6.474569e-01,-4.036508e-01,9.298096e-01
max,974803.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.751404e+00,5.327735e+00,9.524649e+00,9.298096e-01


### Train data

In [22]:
# convert dttm to normalized seconds
stories_train = pd.read_csv('stories_reaction_train.csv')
stories_train['event_dttm'] = pd.to_datetime(stories_train['event_dttm'])
earliest_event = stories_train['event_dttm'].min()

stories_train['event_seconds'] = (stories_train['event_dttm'] - earliest_event)/np.timedelta64(1,'s')
event_mean = np.mean(stories_train['event_seconds'])
event_std = np.std(stories_train['event_seconds'])
stories_train['event_seconds'] = (stories_train['event_seconds'] - event_mean)/event_std

stories_train.drop(columns='event_dttm', inplace=True)

event_score = pd.get_dummies(stories_train['event'])
event_score['customer_id'] = stories_train['customer_id']
event_score['story_id'] = stories_train['story_id']
event_score['event'] = stories_train['event']
#stories_train = stories_train.merge(event_score, how='left', on='customer_id')
# ['dislike', 'like', 'skip', 'view']
event_score.groupby('story_id')[['dislike']].agg('mean')

score = event_score[['customer_id', 'story_id', 'event']]
score = score.merge(event_score.groupby('story_id')[['dislike', 'like', 'skip', 'view']].agg('sum'), 
                    how='left', on='story_id')

score['score'] = pd.Series(np.zeros(len(score)))
'''
for item in tqdm(range(len(score))):
    dislike = score['dislike'].iloc[item]
    like = score['like'].iloc[item]
    skip = score['skip'].iloc[item]
    view = score['view'].iloc[item]
    
    neg = dislike + skip
    pos = like + view
    
    if score['event'].iloc[item] == 'dislike':
        result = -1.0
    elif score['event'].iloc[item] == 'skip':
        result = skip*(-1)/neg
    elif score['event'].iloc[item] == 'view':
        result = view/pos
    elif score['event'].iloc[item] == 'like':
        result = 1.0
        
    score['score'].iloc[item] = result
         
score
'''

"\nfor item in tqdm(range(len(score))):\n    dislike = score['dislike'].iloc[item]\n    like = score['like'].iloc[item]\n    skip = score['skip'].iloc[item]\n    view = score['view'].iloc[item]\n    \n    neg = dislike + skip\n    pos = like + view\n    \n    if score['event'].iloc[item] == 'dislike':\n        result = -1.0\n    elif score['event'].iloc[item] == 'skip':\n        result = skip*(-1)/neg\n    elif score['event'].iloc[item] == 'view':\n        result = view/pos\n    elif score['event'].iloc[item] == 'like':\n        result = 1.0\n        \n    score['score'].iloc[item] = result\n         \nscore\n"

In [23]:
# now merge 

x = stories_train.merge(score[['customer_id', 'story_id', 'score', 'event']], 
                          how='left', on=['customer_id', 'story_id' ,'event'])

x = x.merge(customer_train_merged, how='left', on='customer_id')
x = x.dropna()
x.drop(columns=['customer_id', 'story_id'], inplace=True)
x

,event,event_seconds,score,product_0_CLS,product_0_NU,product_0_UTL,product_1_CLS,product_1_NU,product_1_OPN,product_1_UTL,...,marital_status_cd_CIV,marital_status_cd_DIV,marital_status_cd_DLW,marital_status_cd_MAR,marital_status_cd_UNM,marital_status_cd_WID,seconds,age,children_cnt,job_position_cd
0,view,1.341736,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-3.241953,0.647457,1.251066,0.155072
1,skip,-1.085207,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-3.241953,0.647457,1.251066,0.155072
2,skip,-0.446803,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-3.241953,0.647457,1.251066,0.155072
3,skip,-1.668998,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-3.241953,0.647457,1.251066,0.155072
4,view,1.363281,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-3.241953,0.647457,1.251066,0.155072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507524,view,0.679070,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.006346,0.647457,-0.403651,0.000125
507525,view,-0.936818,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.002921,-1.952698,-0.403651,-0.774613
507526,view,-0.936818,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.002921,-1.952698,-0.403651,-0.774613
507527,like,0.802392,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.002921,-1.952698,-0.403651,-0.774613


In [24]:
y = x['score']
x = x.drop(columns=['score'])

In [25]:
x.to_csv('x_train.csv', index=False)
y.to_csv('y_train.csv', index=False)

### Test data

In [26]:
# convert dttm to normalized seconds
stories_test = pd.read_csv('stories_reaction_test.csv')
stories_test['event_dttm'] = pd.to_datetime(stories_test['event_dttm'])
earliest_event = stories_test['event_dttm'].min()

stories_test['event_seconds'] = (stories_test['event_dttm'] - earliest_event)/np.timedelta64(1,'s')
event_mean = np.mean(stories_test['event_seconds'])
event_std = np.std(stories_test['event_seconds'])
stories_test['event_seconds'] = (stories_test['event_seconds'] - event_mean)/event_std

stories_test.drop(columns='event_dttm', inplace=True)
            
stories_test

,customer_id,story_id,answer_id,event_seconds
0,466906,1152,0,-1.696110
1,466906,1355,1,-1.696110
2,146395,537,2,-1.695957
3,897303,915,3,-1.695916
4,50094,1415,4,-1.695836
...,...,...,...,...
172044,621975,1247,172044,1.832525
172045,621975,1306,172045,1.832525
172046,621975,1537,172046,1.832525
172047,621975,836,172047,1.832525


In [27]:
x_val = stories_test.merge(customer_train_merged, how='left', on='customer_id')
x_val.fillna(value=0.0, inplace=True)
x_val.to_csv('x_val.csv', index=False)